In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import Imputer

import matplotlib.pyplot as plt


import re

from math import sqrt

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.fecha = pd.to_datetime(df.fecha)

In [4]:
df['year'] = df.fecha.dt.year
df['mes'] = df.fecha.dt.month
df['dia_del_mes'] = df.fecha.dt.day
df['dia_del_anio'] = df.fecha.dt.dayofyear
df['quarter'] = df.fecha.dt.quarter

In [5]:
df.quarter.value_counts(normalize = True)

4    0.361492
3    0.245954
2    0.207288
1    0.185267
Name: quarter, dtype: float64

In [6]:
df.dia_del_mes.value_counts(normalize = True)

8     0.038429
3     0.037021
2     0.036554
17    0.036217
25    0.034821
14    0.034629
5     0.034550
6     0.034138
7     0.033771
4     0.033275
1     0.033208
10    0.033113
28    0.032992
24    0.032992
13    0.032933
9     0.032804
27    0.032783
23    0.032654
11    0.032279
18    0.031975
19    0.031846
12    0.031688
22    0.031567
20    0.030521
21    0.030250
26    0.029404
29    0.029212
30    0.029075
16    0.029008
15    0.028242
31    0.018050
Name: dia_del_mes, dtype: float64

In [7]:
df.mes.value_counts(normalize = True)

12    0.175946
10    0.097108
11    0.088438
8     0.087071
9     0.085925
7     0.072958
6     0.071754
4     0.068608
5     0.066925
1     0.066704
3     0.061383
2     0.057179
Name: mes, dtype: float64

In [8]:
df.year.value_counts(normalize = True)

2016    0.391825
2015    0.214458
2014    0.169050
2013    0.126608
2012    0.098058
Name: year, dtype: float64

In [9]:
df.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio,year,mes,dia_del_mes,dia_del_anio,quarter
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,0.0,0.0,0.0,0.0,2273000.0,2015,8,23,235,3
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,0.0,0.0,1.0,1.0,3600000.0,2013,6,28,179,2
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,0.0,0.0,0.0,0.0,1200000.0,2015,10,17,290,4
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,0.0,0.0,1.0,1.0,650000.0,2012,3,9,69,1
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,0.0,0.0,0.0,0.0,1150000.0,2016,6,7,159,2


In [10]:
df.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio', 'year', 'mes', 'dia_del_mes',
       'dia_del_anio', 'quarter'],
      dtype='object')

In [11]:
df.shape

(240000, 28)

In [12]:
df.isnull().sum()

id                                 0
titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
year                               0
mes                                0
dia_del_mes                        0
dia_del_anio                       0
q

In [13]:
df.tipodepropiedad.value_counts(normalize = True)

Casa                             0.590601
Apartamento                      0.238967
Casa en condominio               0.080420
Terreno                          0.041445
Local Comercial                  0.012732
Oficina comercial                0.007256
Bodega comercial                 0.005859
Edificio                         0.005818
Terreno comercial                0.005526
Casa uso de suelo                0.002951
Quinta Vacacional                0.001646
Duplex                           0.001429
Villa                            0.001417
Inmuebles productivos urbanos    0.000833
Rancho                           0.000708
Local en centro comercial        0.000688
Departamento Compartido          0.000588
Otros                            0.000558
Nave industrial                  0.000317
Terreno industrial               0.000129
Huerta                           0.000083
Lote                             0.000021
Hospedaje                        0.000004
Garage                           0

In [14]:
df.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio', 'year', 'mes', 'dia_del_mes',
       'dia_del_anio', 'quarter'],
      dtype='object')

In [15]:
mean_met = df.groupby(df.tipodepropiedad).agg({'metroscubiertos': 'mean','metrostotales': 'mean','gimnasio':'mean','usosmultiples':'mean','piscina':'mean','escuelascercanas':'mean','centroscomercialescercanos':'mean','garages':'mean', 'banos':'mean'})

In [16]:
type(mean_met)

pandas.core.frame.DataFrame

In [17]:
mean_met.metroscubiertos.fillna(0, inplace=True)
mean_met.banos.fillna(0,inplace=True)

In [18]:
mean_met.head()

,metroscubiertos,metrostotales,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,garages,banos
tipodepropiedad,,,,,,,,,
Apartamento,117.048657,117.907762,0.132383,0.107898,0.141173,0.458921,0.450847,1.410242,1.816817
Bodega comercial,251.153639,261.715040,0.000000,0.000000,0.000000,0.156472,0.168563,0.484677,1.847185
Casa,194.026911,183.767889,0.043079,0.035980,0.064276,0.455506,0.385564,1.705926,2.240429
Casa en condominio,187.697231,169.728656,0.065399,0.098824,0.154065,0.498730,0.459605,1.908157,2.366065
Casa uso de suelo,254.068966,228.425710,0.000000,0.000000,0.000000,0.326271,0.316384,1.427350,2.404719


In [19]:
mean_met = mean_met.rename(columns = {"metroscubiertos": "mean_metroscubiertos", "metrostotales": "mean_metrostotales","gimnasio":"mean_gimnasio", "usosmultiples":"mean_usosmultiples","piscina":"mean_piscina","escuelascercanas":"mean_escuelascercanas","centroscomercialescercanos":"mean_centroscomercialescercanos","garages":"mean_garages","banos":"mean_banos"})

In [20]:
mean_met.reset_index()

,tipodepropiedad,mean_metroscubiertos,mean_metrostotales,mean_gimnasio,mean_usosmultiples,mean_piscina,mean_escuelascercanas,mean_centroscomercialescercanos,mean_garages,mean_banos
0,Apartamento,117.048657,117.907762,0.132383,0.107898,0.141173,0.458921,0.450847,1.410242,1.816817
1,Bodega comercial,251.153639,261.715040,0.000000,0.000000,0.000000,0.156472,0.168563,0.484677,1.847185
2,Casa,194.026911,183.767889,0.043079,0.035980,0.064276,0.455506,0.385564,1.705926,2.240429
3,Casa en condominio,187.697231,169.728656,0.065399,0.098824,0.154065,0.498730,0.459605,1.908157,2.366065
4,Casa uso de suelo,254.068966,228.425710,0.000000,0.000000,0.000000,0.326271,0.316384,1.427350,2.404719
5,Departamento Compartido,127.654676,130.405941,0.241135,0.184397,0.184397,0.609929,0.666667,1.401460,1.791045
6,Duplex,100.215976,100.283019,0.000000,0.000000,0.000000,0.011662,0.008746,1.181548,1.360947
7,Edificio,291.231944,231.461169,0.000000,0.000716,0.000716,0.305874,0.313754,0.725086,2.626703
8,Garage,40.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Hospedaje,288.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
mean_met.shape

(24, 9)

In [22]:
df.shape

(240000, 28)

In [23]:
df = pd.merge(df, mean_met, on='tipodepropiedad', how='outer')

In [24]:
df.shape

(240000, 37)

In [25]:
df.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio', 'year', 'mes', 'dia_del_mes',
       'dia_del_anio', 'quarter', 'mean_metroscubiertos', 'mean_metrostotales',
       'mean_gimnasio', 'mean_usosmultiples', 'mean_piscina',
       'mean_escuelascercanas', 'mean_centroscomercialescercanos',
       'mean_garages', 'mean_banos'],
      dtype='object')

In [26]:
df.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,quarter,mean_metroscubiertos,mean_metrostotales,mean_gimnasio,mean_usosmultiples,mean_piscina,mean_escuelascercanas,mean_centroscomercialescercanos,mean_garages,mean_banos
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,3,117.048657,117.907762,0.132383,0.107898,0.141173,0.458921,0.450847,1.410242,1.816817
1,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,2,117.048657,117.907762,0.132383,0.107898,0.141173,0.458921,0.450847,1.410242,1.816817
2,126147,departamento en venta taxqueña,"amplio departamento, estancia de sala y comedo...",Apartamento,Condominio Tlalpan 2B,Coyoacán,Distrito Federal,5.0,2.0,1.0,...,1,117.048657,117.907762,0.132383,0.107898,0.141173,0.458921,0.450847,1.410242,1.816817
3,44962,pre- venta preciosos depas 2 recamaras con sub...,<p>pre-venta de preciosos departamento ecologi...,Apartamento,BUENAVISTA DEPTOS CON SUBSIDIO,Villa de Alvarez,Colima,1.0,2.0,1.0,...,1,117.048657,117.907762,0.132383,0.107898,0.141173,0.458921,0.450847,1.410242,1.816817
4,103293,"departamento en venta, san pedro garza garcia,...","departamento nuevo ,256 m2 de construccion,un ...",Apartamento,NaN,San Pedro Garza García,Nuevo León,NaN,3.0,2.0,...,4,117.048657,117.907762,0.132383,0.107898,0.141173,0.458921,0.450847,1.410242,1.816817


In [27]:
df['metro_promedio_por_habitacion'] = df.metroscubiertos / (df.habitaciones + df.banos +1 - df.garages) #El 1 representa el ambiente en comun

In [28]:
df['aire_libre'] = df.metrostotales - df.metroscubiertos

In [29]:
df.shape

(240000, 39)

In [30]:
filterCol = ['id','titulo','descripcion','direccion','ciudad','idzona','lat','lng','fecha']

In [31]:
X = df.drop(['precio'] + filterCol,axis = 1)
y = df['precio']

In [32]:
X.head()

,tipodepropiedad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,gimnasio,usosmultiples,...,mean_metrostotales,mean_gimnasio,mean_usosmultiples,mean_piscina,mean_escuelascercanas,mean_centroscomercialescercanos,mean_garages,mean_banos,metro_promedio_por_habitacion,aire_libre
0,Apartamento,Distrito Federal,NaN,2.0,1.0,2.0,80.0,80.0,0.0,0.0,...,117.907762,0.132383,0.107898,0.141173,0.458921,0.450847,1.410242,1.816817,20.000000,0.0
1,Apartamento,Jalisco,10.0,2.0,1.0,1.0,95.0,95.0,0.0,0.0,...,117.907762,0.132383,0.107898,0.141173,0.458921,0.450847,1.410242,1.816817,31.666667,0.0
2,Apartamento,Distrito Federal,5.0,2.0,1.0,1.0,75.0,90.0,0.0,0.0,...,117.907762,0.132383,0.107898,0.141173,0.458921,0.450847,1.410242,1.816817,25.000000,15.0
3,Apartamento,Colima,1.0,2.0,1.0,1.0,58.0,NaN,0.0,0.0,...,117.907762,0.132383,0.107898,0.141173,0.458921,0.450847,1.410242,1.816817,19.333333,NaN
4,Apartamento,Nuevo León,NaN,3.0,2.0,4.0,256.0,NaN,0.0,0.0,...,117.907762,0.132383,0.107898,0.141173,0.458921,0.450847,1.410242,1.816817,42.666667,NaN


In [33]:
X.shape

(240000, 29)

In [34]:
X = pd.get_dummies(X, dummy_na=False)

In [35]:
X.shape

(240000, 83)

In [36]:
y.shape

(240000,)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=7)

In [38]:
parametros = {'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_depth': 10,
 'min_child_weight': 1,
 'n_estimators': 100,
 'nthread': 4,
 'random_state': 0,
 'subsample': 1,
 #'eval_metric' : 'mae'             
}

In [39]:
model = XGBRegressor(**parametros)

In [40]:
%%time
model.fit(X_train, y_train)

/home/braianvillalba/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[13:54:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 1min 48s, sys: 2.56 s, total: 1min 51s
Wall time: 27.9 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=4, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [41]:
pred = model.predict(X_test)

In [42]:
len(pred)

60000

In [43]:
mean_absolute_error(y_test,pred)

643890.8844751302

#### without metrics in hiper

In [44]:
parametros2 = {
'objective':'reg:gamma',
'colsample_bytree': 0.8,
'gamma': 0,
'learning_rate': 0.1,
'max_depth': 10,
'min_child_weight': 1,
'n_estimators': 1000,
'nthread': 4,
'random_state': 0,
'subsample': 1,
'eval_metric' : 'mae'
            
}

In [45]:
model2 = XGBRegressor(**parametros2)

In [46]:
%%time
model2.fit(X_train, y_train)

/home/braianvillalba/.local/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


CPU times: user 18min 1s, sys: 17.4 s, total: 18min 18s
Wall time: 4min 35s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eval_metric='mae',
             gamma=0, importance_type='gain', learning_rate=0.1,
             max_delta_step=0, max_depth=10, min_child_weight=1, missing=None,
             n_estimators=1000, n_jobs=1, nthread=4, objective='reg:gamma',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [47]:
pred2 = model2.predict(X_test)

In [48]:
len(pred2)

60000

In [49]:
pred2[:4]

array([ 701055.56, 2477766.2 , 2520556.5 , 5248483.5 ], dtype=float32)

In [50]:
len(y_test)

60000

In [51]:
y_test[:4]

191056     690000.0
108467    2290000.0
8889      2990000.0
47055     4206500.0
Name: precio, dtype: float64

In [52]:
mean_absolute_error(y_test,pred2)

626604.2074635417

In [53]:
df_submit = pd.read_csv('test.csv')

In [54]:
df_submit.shape

(60000, 22)

In [55]:
dataSubmit = pd.DataFrame()
dataSubmit['id'] = df_submit['id']
dataSubmit['target'] = pred2

In [56]:
dataSubmit.to_csv("data/workshop-xgboost-v4_1.csv", index=False, header = True)